In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [7]:
pyspark.__version__

'3.5.5'

In [8]:
spark.version

'3.5.5'

In [10]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [12]:
df.show(3)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [16]:
df \
.repartition(4) \
.write.parquet('data/yellow/2024/10', mode='overwrite')

In [17]:
! ls -lh data/yellow/2024/10

total 90M
-rw-r--r-- 1 maxkaizo maxkaizo   0 Mar  7 15:19 _SUCCESS
-rw-r--r-- 1 maxkaizo maxkaizo 23M Mar  7 15:19 part-00000-ed178c3b-2821-4ad4-8454-93ba2f6b80da-c000.snappy.parquet
-rw-r--r-- 1 maxkaizo maxkaizo 23M Mar  7 15:19 part-00001-ed178c3b-2821-4ad4-8454-93ba2f6b80da-c000.snappy.parquet
-rw-r--r-- 1 maxkaizo maxkaizo 23M Mar  7 15:19 part-00002-ed178c3b-2821-4ad4-8454-93ba2f6b80da-c000.snappy.parquet
-rw-r--r-- 1 maxkaizo maxkaizo 23M Mar  7 15:19 part-00003-ed178c3b-2821-4ad4-8454-93ba2f6b80da-c000.snappy.parquet


In [19]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [20]:
df.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 892962|
|       2|2940734|
|       6|     75|
+--------+-------+



In [21]:
df.registerTempTable('trips_data')

/home/maxkaizo/spark/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
spark.sql("""
SELECT
    count(1)
FROM
    trips_data
WHERE
    CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [26]:
spark.sql("""
SELECT
    max(trip_distance)
FROM
    trips_data
""").show()

+------------------+
|max(trip_distance)|
+------------------+
|         366343.04|
+------------------+



In [22]:
spark.sql("""
SELECT
    VendorID,
    count(1)
FROM
    trips_data
GROUP BY
    VendorID
""").show()

+--------+--------+
|VendorID|count(1)|
+--------+--------+
|       1|  892962|
|       2| 2940734|
|       6|      75|
+--------+--------+



In [ ]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [29]:
from pyspark.sql import functions as F

In [30]:
df.head(2)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 12, 20), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 25, 25), passenger_count=1, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=236, payment_type=1, fare_amount=14.2, extra=3.5, mta_tax=0.5, tip_amount=3.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=23.0, congestion_surcharge=2.5, Airport_fee=0.0)]

In [31]:
df.withColumn('trip_duration', F.timestamp_diff('HOUR', tpep_pickup_datetime, tpep_dropoff_datetime ))

AttributeError: module 'pyspark.sql.functions' has no attribute 'timestamp_diff'